In [ ]:
import os
import re
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics.pairwise import haversine_distances
from tqdm import tqdm

# --- Constants ---
HISTORY_LEN = 48
PRED_HORIZON = 4
K_NEIGHBORS = 2
VAL_RATIO = 0.15
HOLDOUT_RATIO = 0.15
EPOCHS = 50
PATIENCE = 8
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BEST_PARAMS = {
    "hidden_dim": 32,
    "dropout": 0.3,
    "lr": 0.001,
    "weight_decay": 1e-5,
    "batch_size": 64
}

# --- Helpers ---
def clean_name(name):
    return re.sub(r'\W+', '_', name.strip())

# --- Load data ---
df = pd.read_csv('bicikelj_train.csv')
test_df = pd.read_csv('bicikelj_test.csv')
meta = pd.read_csv('bicikelj_metadata.csv')
station_cols = df.columns[1:]

for col in station_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df[station_cols] = df[station_cols].ffill().bfill()
df = df.dropna(subset=station_cols, how='all').reset_index(drop=True)

# --- Neighbors ---
coords = np.deg2rad(meta[['latitude', 'longitude']].values)
station_names = meta['name'].tolist()
dists = haversine_distances(coords, coords) * 6371
neighbors = {}
for i, name in enumerate(station_names):
    order = np.argsort(dists[i])
    nn_idx = [j for j in order if j != i][:K_NEIGHBORS]
    neighbors[name] = [station_names[j] for j in nn_idx]

# --- Dataset ---
class BicikeljDataset(Dataset):
    def __init__(self, df, station_name, station_cols, neighbors, history_len, pred_horizon):
        self.samples = []
        timestamps = pd.to_datetime(df['timestamp'])
        hours = (timestamps.dt.hour / 23.0).values
        dows = (timestamps.dt.dayofweek / 6.0).values
        bikes = df[station_cols].values.astype(np.float32)
        name_to_idx = {name: i for i, name in enumerate(station_cols)}
        s_idx = name_to_idx[station_name]
        nn_idx = [name_to_idx[nn] for nn in neighbors[station_name]]

        for i in range(history_len, len(df) - pred_horizon + 1):
            seq = []
            for t in range(i - history_len, i):
                row = [bikes[t, s_idx]]
                row += [bikes[t, j] for j in nn_idx]
                row += [hours[t], dows[t]]
                seq.append(row)
            seq = np.stack(seq)
            target = bikes[i:i + pred_horizon, s_idx]
            self.samples.append((seq, target))

    def __len__(self): return len(self.samples)
    def __getitem__(self, idx):
        x, y = self.samples[idx]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

# --- Model ---
class LSTMForecast(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        _, (h_n, _) = self.lstm(x)
        return self.fc(self.dropout(h_n[-1]))

# --- Training ---
def train_lstm(model, train_loader, val_loader, config, device):
    optimizer = torch.optim.Adam(model.parameters(), lr=config["lr"], weight_decay=config["weight_decay"])
    criterion = nn.MSELoss()
    best_loss = float('inf')
    best_state = None
    patience_counter = 0

    for epoch in range(EPOCHS):
        model.train()
        epoch_loss = 0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            loss = criterion(model(xb), yb)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                val_loss += criterion(model(xb), yb).item()

        print(f"Epoch {epoch+1}: Train Loss = {epoch_loss / len(train_loader):.4f}, Val Loss = {val_loss / len(val_loader):.4f}")

        if val_loss < best_loss:
            best_loss = val_loss
            best_state = model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= PATIENCE:
                print("Early stopping.")
                break

    model.load_state_dict(best_state)
    return model

# --- Train models ---
os.makedirs("models", exist_ok=True)
existing_models = set(os.listdir("models"))

for station in tqdm(station_cols, desc="Training per station"):
    safe_name = clean_name(station)
    model_filename = f"lstm_model_{safe_name}.pt"
    model_path = os.path.join("models", model_filename)

    if model_filename in existing_models:
        print(f"Model already exists for {station}, skipping.")
        continue

    dataset = BicikeljDataset(df, station, station_cols, neighbors, HISTORY_LEN, PRED_HORIZON)
    total = len(dataset)
    train_end = int(total * (1 - VAL_RATIO - HOLDOUT_RATIO))
    val_end = int(total * (1 - HOLDOUT_RATIO))

    train_set = torch.utils.data.Subset(dataset, range(0, train_end))
    val_set = torch.utils.data.Subset(dataset, range(train_end, val_end))
    train_loader = DataLoader(train_set, batch_size=BEST_PARAMS["batch_size"], shuffle=True)
    val_loader = DataLoader(val_set, batch_size=BEST_PARAMS["batch_size"])

    sample_x, sample_y = dataset[0]
    input_dim = sample_x.shape[1]
    output_dim = sample_y.shape[0]

    model = LSTMForecast(input_dim, BEST_PARAMS["hidden_dim"], output_dim, BEST_PARAMS["dropout"]).to(DEVICE)
    model = train_lstm(model, train_loader, val_loader, BEST_PARAMS, DEVICE)
    torch.save(model.state_dict(), model_path)

# --- Predict missing values in test set ---
test_feats = test_df[station_cols].values.astype(np.float32)
timestamps = pd.to_datetime(test_df["timestamp"])
hours = (timestamps.dt.hour / 23.0).values
dows = (timestamps.dt.dayofweek / 6.0).values
name_to_idx = {name: i for i, name in enumerate(station_cols)}

pred_matrix = np.full_like(test_feats, np.nan)

with torch.no_grad():
    for i in range(HISTORY_LEN, len(test_df) - PRED_HORIZON + 1):
        if np.isnan(test_feats[i:i + PRED_HORIZON]).all(axis=0).all():
            for station in station_cols:
                s_idx = name_to_idx[station]
                nn_idx = [name_to_idx[nn] for nn in neighbors[station]]

                seq = []
                for t in range(i - HISTORY_LEN, i):
                    row = [test_feats[t, s_idx]]
                    row += [test_feats[t, j] for j in nn_idx]
                    row += [hours[t], dows[t]]
                    seq.append(row)
                seq = torch.tensor([seq], dtype=torch.float32).to(DEVICE)

                input_dim = 1 + K_NEIGHBORS + 2
                output_dim = PRED_HORIZON
                model = LSTMForecast(input_dim, BEST_PARAMS["hidden_dim"], output_dim, BEST_PARAMS["dropout"]).to(DEVICE)
                model_path = f"models/lstm_model_{clean_name(station)}.pt"
                model.load_state_dict(torch.load(model_path))
                model.eval()

                pred = model(seq).cpu().numpy().flatten()
                for j in range(PRED_HORIZON):
                    pred_matrix[i + j, s_idx] = pred[j]

pred_df = pd.DataFrame(pred_matrix, columns=station_cols)
pred_df.insert(0, "timestamp", test_df["timestamp"])
rows_to_output = test_df[station_cols].isna().all(axis=1)
pred_df_filtered = pred_df[rows_to_output].copy()
pred_df_filtered.to_csv("bicikelj_test_predictions.csv", index=False)

Training per station:   0%|          | 0/84 [00:00<?, ?it/s]

Epoch 1: Train Loss = 29.7295, Val Loss = 20.9033
Epoch 2: Train Loss = 15.6793, Val Loss = 15.4392
Epoch 3: Train Loss = 13.4229, Val Loss = 13.8457
Epoch 4: Train Loss = 12.6746, Val Loss = 13.1875
Epoch 5: Train Loss = 12.3440, Val Loss = 12.8634
Epoch 6: Train Loss = 12.2439, Val Loss = 12.7978
Epoch 7: Train Loss = 12.1317, Val Loss = 12.6497
Epoch 8: Train Loss = 11.9795, Val Loss = 12.4275
Epoch 9: Train Loss = 11.8643, Val Loss = 12.3524
Epoch 10: Train Loss = 11.7596, Val Loss = 12.3491
Epoch 11: Train Loss = 11.6362, Val Loss = 12.2196
Epoch 12: Train Loss = 11.5406, Val Loss = 12.0575
Epoch 13: Train Loss = 11.3793, Val Loss = 12.0079
Epoch 14: Train Loss = 11.3727, Val Loss = 11.9949
Epoch 15: Train Loss = 11.3154, Val Loss = 11.8741
Epoch 16: Train Loss = 11.3314, Val Loss = 11.8978
Epoch 17: Train Loss = 11.2525, Val Loss = 11.8609
Epoch 18: Train Loss = 11.1421, Val Loss = 11.8894
Epoch 19: Train Loss = 11.1697, Val Loss = 11.9079
Epoch 20: Train Loss = 11.0645, Val Loss

Training per station:   1%|          | 1/84 [00:41<57:35, 41.63s/it]

Epoch 42: Train Loss = 10.3155, Val Loss = 11.6155
Early stopping.
Epoch 1: Train Loss = 42.9113, Val Loss = 17.5829
Epoch 2: Train Loss = 15.9180, Val Loss = 10.9767
Epoch 3: Train Loss = 11.1957, Val Loss = 9.8316
Epoch 4: Train Loss = 9.7984, Val Loss = 9.3024
Epoch 5: Train Loss = 9.2825, Val Loss = 9.0341
Epoch 6: Train Loss = 8.9736, Val Loss = 8.9396
Epoch 7: Train Loss = 8.8208, Val Loss = 8.8617
Epoch 8: Train Loss = 8.6298, Val Loss = 8.8162
Epoch 9: Train Loss = 8.6164, Val Loss = 9.0395
Epoch 10: Train Loss = 8.5313, Val Loss = 8.8262
Epoch 11: Train Loss = 8.4556, Val Loss = 8.8258
Epoch 12: Train Loss = 8.3380, Val Loss = 8.6815
Epoch 13: Train Loss = 8.2858, Val Loss = 8.7142
Epoch 14: Train Loss = 8.2883, Val Loss = 8.6704
Epoch 15: Train Loss = 8.2354, Val Loss = 8.6880
Epoch 16: Train Loss = 8.1165, Val Loss = 8.6319
Epoch 17: Train Loss = 8.0627, Val Loss = 8.8497
Epoch 18: Train Loss = 8.1668, Val Loss = 8.6727
Epoch 19: Train Loss = 8.0893, Val Loss = 8.7398
Epoch 

Training per station:   2%|▏         | 2/84 [01:16<51:38, 37.78s/it]

...

Epoch 42: Train Loss = 8.6287, Val Loss = 7.4180
Early stopping.
Epoch 1: Train Loss = 82.8907, Val Loss = 50.1593
Epoch 2: Train Loss = 27.4301, Val Loss = 22.4590
Epoch 3: Train Loss = 14.9733, Val Loss = 14.4382
Epoch 4: Train Loss = 11.5687, Val Loss = 11.5252
Epoch 5: Train Loss = 10.2696, Val Loss = 10.2668
Epoch 6: Train Loss = 9.7465, Val Loss = 9.6196
Epoch 7: Train Loss = 9.4283, Val Loss = 9.3380
Epoch 8: Train Loss = 9.0760, Val Loss = 8.8489
Epoch 9: Train Loss = 8.8972, Val Loss = 8.8008
Epoch 10: Train Loss = 8.6793, Val Loss = 8.4192
Epoch 11: Train Loss = 8.5660, Val Loss = 8.3805
Epoch 12: Train Loss = 8.4993, Val Loss = 8.3161
Epoch 13: Train Loss = 8.3282, Val Loss = 8.2419
Epoch 14: Train Loss = 8.2077, Val Loss = 7.9995
Epoch 15: Train Loss = 8.1481, Val Loss = 8.1916
Epoch 16: Train Loss = 8.1393, Val Loss = 8.0904
Epoch 17: Train Loss = 8.0089, Val Loss = 7.9049
Epoch 18: Train Loss = 7.9561, Val Loss = 7.7594
Epoch 19: Train Loss = 7.8581, Val Loss = 7.6889
Epo

Training per station:  99%|█████████▉| 83/84 [47:25<00:36, 36.08s/it]

Epoch 38: Train Loss = 7.2863, Val Loss = 7.8210
Early stopping.
Epoch 1: Train Loss = 32.6387, Val Loss = 22.4304
Epoch 2: Train Loss = 15.0511, Val Loss = 14.5331
Epoch 3: Train Loss = 12.1749, Val Loss = 12.5080
Epoch 4: Train Loss = 11.5283, Val Loss = 11.6965
Epoch 5: Train Loss = 11.1087, Val Loss = 11.5564
Epoch 6: Train Loss = 10.8406, Val Loss = 11.0998
Epoch 7: Train Loss = 10.8768, Val Loss = 10.9942
Epoch 8: Train Loss = 10.6876, Val Loss = 10.9237
Epoch 9: Train Loss = 10.6738, Val Loss = 10.8534
Epoch 10: Train Loss = 10.5467, Val Loss = 10.9884
Epoch 11: Train Loss = 10.4859, Val Loss = 10.7414
Epoch 12: Train Loss = 10.4415, Val Loss = 10.6526
Epoch 13: Train Loss = 10.3470, Val Loss = 10.7907
Epoch 14: Train Loss = 10.3359, Val Loss = 10.5565
Epoch 15: Train Loss = 10.2115, Val Loss = 10.4430
Epoch 16: Train Loss = 10.1429, Val Loss = 10.5259
Epoch 17: Train Loss = 10.0359, Val Loss = 10.3475
Epoch 18: Train Loss = 10.0141, Val Loss = 10.4730
Epoch 19: Train Loss = 10.

Training per station: 100%|██████████| 84/84 [47:56<00:00, 34.24s/it]

Epoch 32: Train Loss = 9.4381, Val Loss = 10.5628
Early stopping.


In [ ]:
!zip -r models.zip models/


  adding: models/ (stored 0%)
  adding: models/lstm_model_TEHNOLOŠKI_PARK.pt (deflated 11%)
  adding: models/lstm_model_LIDL_RUDNIK.pt (deflated 12%)
  adding: models/lstm_model_BAVARSKI_DVOR.pt (deflated 12%)
  adding: models/lstm_model_POGAČARJEV_TRG_TRŽNICA.pt (deflated 11%)
  adding: models/lstm_model_TRŽNICA_KOSEZE.pt (deflated 11%)
  adding: models/lstm_model_PRUŠNIKOVA.pt (deflated 11%)
  adding: models/lstm_model_VIŠKO_POLJE.pt (deflated 11%)
  adding: models/lstm_model_TRNOVO.pt (deflated 12%)
  adding: models/lstm_model_AMBROŽEV_TRG.pt (deflated 11%)
  adding: models/lstm_model_PARK_NAVJE_ŽELEZNA_CESTA.pt (deflated 11%)
  adding: models/lstm_model_BRATOVŠEVA_PLOŠČAD.pt (deflated 11%)
  adding: models/lstm_model_KOPRSKA_ULICA.pt (deflated 12%)
  adding: models/lstm_model_KOPALIŠČE_KOLEZIJA.pt (deflated 11%)
  adding: models/lstm_model_CITYPARK.pt (deflated 12%)
  adding: models/lstm_model_HOFER_KAJUHOVA.pt (deflated 12%)
  adding: models/lstm_model_LEK_VEROVŠKOVA.pt (deflated 